In [123]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import itertools

In [117]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1239,tt3063516,15000000,151831537,Jackass Presents: Bad Grandpa,Johnny Knoxville|Jackson Nicoll|Georgina Cates...,Jeff Tremaine,Real people. Real reactions. Real messed up.,86-year-old Irving Zisman is on a journey acro...,92,Comedy,Paramount Pictures|MTV Films|Dickhouse Product...,10/22/2013,6.1,2013
1214,tt0790736,130000000,61648500,R.I.P.D.,Jeff Bridges|Ryan Reynolds|Kevin Bacon|Stephan...,Robert Schwentke,To protect and serve the living,A recently slain cop joins a team of undead po...,96,Fantasy|Action|Comedy|Crime,Universal Pictures|Original Film|Dark Horse En...,7/18/2013,5.3,2013
741,tt1216475,200000000,559852396,Cars 2,Larry the Cable Guy|Owen Wilson|Michael Caine|...,John Lasseter|Brad Lewis,Ka-ciao!,Star race car Lightning McQueen and his pal Ma...,106,Animation|Family|Adventure|Comedy,Walt Disney Pictures|Pixar Animation Studios,6/11/2011,5.8,2011
1171,tt0338466,55000000,33828318,Stuck on You,Matt Damon|Greg Kinnear|Eva Mendes|Wen Yann Sh...,Bobby Farrelly|Peter Farrelly,"Whatever happens to the boys, one thing is cer...","In Martha's Vineyard, Mass., conjoined twins W...",118,Comedy,Twentieth Century Fox Film Corporation|Conundr...,12/10/2003,4.8,2003
1497,tt0457510,32000000,80197993,Nacho Libre,Jack Black|Ana de la Reguera|HÃ©ctor JimÃ©nez|...,Jared Hess,He's not lean. He's not mean. He's nacho avera...,Nacho Libre is loosely based on the story of F...,92,Comedy|Family,Paramount Pictures|Nickelodeon Movies|Black & ...,6/16/2006,5.5,2006


In [54]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

# Предобработка

In [210]:
answers = {}  # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

# the time given in the dataset is in string format.
# So we need to change this in datetime format
# ...

#import datetime
#data['release_date'] = data['release_date'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y'))
data['release_date'] = pd.to_datetime(data['release_date'])
data['profit'] = data['revenue'] - data['budget']
data['genres_array'] = data['genres'].str.split('|')
data['director_array'] = data['director'].str.split('|')
data['cast_array'] = data['cast'].str.split('|')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,genres_array,director_array,cast_array
1209,tt1335975,175000000,150962475,47 Ronin,Keanu Reeves|Hiroyuki Sanada|Kou Shibasaki|Tad...,Carl Rinsch,For courage. For loyalty. For honour.,"Based on the original 1941 movie from Japan, a...",118,Action|Adventure|Fantasy,Mid Atlantic Films|Stuber Productions|Universa...,2013-12-06,5.8,2013,-24037525,"[Action, Adventure, Fantasy]",[Carl Rinsch],"[Keanu Reeves, Hiroyuki Sanada, Kou Shibasaki,..."
61,tt3152624,35000000,140795793,Trainwreck,Amy Schumer|Bill Hader|Brie Larson|Colin Quinn...,Judd Apatow,We All Know One.,Having thought that monogamy was never possibl...,125,Comedy,Universal Pictures|Fuji Television Network|Den...,2015-07-17,6.1,2015,105795793,[Comedy],[Judd Apatow],"[Amy Schumer, Bill Hader, Brie Larson, Colin Q..."
662,tt0421073,20000000,65569869,Street Kings,Keanu Reeves|Forest Whitaker|Chris Evans|Hugh ...,David Ayer,Their city. Their rules. No prisoners.,Tom Ludlow is a disillusioned L.A. Police Offi...,109,Action|Crime|Drama|Thriller,Fox Searchlight Pictures|Regency Enterprises|D...,2008-04-10,6.2,2008,45569869,"[Action, Crime, Drama, Thriller]",[David Ayer],"[Keanu Reeves, Forest Whitaker, Chris Evans, H..."
741,tt1216475,200000000,559852396,Cars 2,Larry the Cable Guy|Owen Wilson|Michael Caine|...,John Lasseter|Brad Lewis,Ka-ciao!,Star race car Lightning McQueen and his pal Ma...,106,Animation|Family|Adventure|Comedy,Walt Disney Pictures|Pixar Animation Studios,2011-06-11,5.8,2011,359852396,"[Animation, Family, Adventure, Comedy]","[John Lasseter, Brad Lewis]","[Larry the Cable Guy, Owen Wilson, Michael Cai..."
1799,tt0172495,103000000,457640427,Gladiator,Russell Crowe|Joaquin Phoenix|Connie Nielsen|O...,Ridley Scott,A Hero Will Rise.,General Maximus' success in battle earns the f...,155,Action|Drama|Adventure,DreamWorks SKG|Universal Pictures|Scott Free P...,2000-05-01,7.7,2000,354640427,"[Action, Drama, Adventure]",[Ridley Scott],"[Russell Crowe, Joaquin Phoenix, Connie Nielse..."


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [88]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример:
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [61]:
# тут пишем ваш код для решения данного вопроса:
line = data[data['budget'] == data['budget'].max()]
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

ВАРИАНТ 2

In [0]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [0]:
# думаю логику работы с этим словарем вы уже поняли,
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [62]:
line = data[data['runtime'] == data['runtime'].max()]
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [89]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

In [90]:
line = data[data['runtime'] == data['runtime'].min()]
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [91]:
answers['4'] = '2. 110'

In [92]:
int(round(data['runtime'].mean(), 0))

110

# 5. Каково медианное значение длительности фильмов? 

In [93]:
answers['5'] = '1. 107'

In [94]:
int(round(data['runtime'].agg(['median']), 0))

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [96]:
# лучше код получения столбца profit вынести в Предобработку что в начале
answers['6'] = '5. Avatar (tt0499549)'

In [95]:
line = data[data['profit'] == data['profit'].max()]
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [97]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

In [98]:
line = data[data['profit'] == data['profit'].min()]
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [99]:
answers['8'] = '1. 1478'

In [100]:
len(data[data['profit'] > 0])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [101]:
answers['9'] = '4. The Dark Knight (tt0468569)'

In [102]:
line_id = data.query('release_year == 2008')['revenue'].idxmax()
f"{data.iloc[line_id]['original_title']} ({data.iloc[line_id]['imdb_id']})"

'The Dark Knight (tt0468569)'

Вариант 2

In [83]:
line = data[data['release_year'] == 2008].sort_values(
    'revenue', ascending=False)
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'The Dark Knight (tt0468569)'

Вариант 3

In [84]:
line = data.query('release_year == 2008').query('revenue == revenue.max()')
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [103]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

In [104]:
line_id = data.query('release_year >= 2012 and release_year <= 2014')[
    'profit'].idxmin()
f"{data.iloc[line_id]['original_title']} ({data.iloc[line_id]['imdb_id']})"

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [150]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

answers['11'] = '3. Drama'

In [139]:
data['genres_array'].explode().value_counts().head(1)

Drama    782
Name: genres_array, dtype: int64

Вариант 2

In [147]:
data['genres'].str.split('|', expand=True).stack().value_counts().head(1)

Drama    782
dtype: int64

Вариант 3

In [146]:
pd.Series(itertools.chain.from_iterable(
    data['genres_array'])).value_counts().head(1)

Drama    782
dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [149]:
answers['12'] = '1. Drama'

In [148]:
data.query('profit > 0')['genres_array'].explode().value_counts().head(1)

Drama    560
Name: genres_array, dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [169]:
answers['13'] = '5. Peter Jackson'

In [201]:
data[['director_array', 'revenue']].explode('director_array').groupby('director_array').sum()\
    .sort_values('revenue', ascending=False).head(1)

,revenue
director_array,
Peter Jackson,6490593685


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [194]:
answers['14'] = '3. Robert Rodriguez'

Вариант 1

In [208]:
data[['director_array', 'genres_array']].explode('director_array').explode('genres_array')\
    .query('genres_array == "Action"')['director_array'].value_counts().head(1)

Robert Rodriguez    9
Name: director_array, dtype: int64

Вариант 2

In [209]:
data[data['genres'].str.contains(
    'Action')]['director_array'].explode().value_counts().head(1)

Robert Rodriguez    9
Name: director_array, dtype: int64

Вариант 3

In [216]:
directors = Counter(data[data['genres'].str.contains(
    'Action')]['director_array'].explode())
directors.most_common(1)

[('Robert Rodriguez', 9)]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [213]:
answers['15'] = '3. Chris Hemsworth'

In [214]:
data.query('release_year == 2012')[['cast_array', 'revenue']].explode('cast_array').groupby('cast_array').sum()\
    .sort_values('revenue', ascending=False).head(1)

,revenue
cast_array,
Chris Hemsworth,2027450773


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [227]:
answers['16'] = '3. Matt Damon'

In [229]:
mean_budget = data['budget'].mean()
data[data['budget'] > mean_budget]['cast_array'].explode().value_counts().head(1)

Matt Damon    18
Name: cast_array, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [230]:
answers['17'] = '2. Action'

Вариант 1

In [231]:
data[data['cast'].str.contains(
    'Nicolas Cage')]['genres_array'].explode().value_counts().head(1)

Action    17
Name: genres_array, dtype: int64

Вариант 2 (с защитой от актеров с похожим именем, например Nicolas Cagesss)

In [240]:
data[['cast_array', 'genres_array']].explode('cast_array').query(
    'cast_array == "Nicolas Cage"')['genres_array'].explode().value_counts().head(1)

Action    17
Name: genres_array, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

# 20. Какой самый прибыльный год для студии Warner Bros?

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

# 23. Для какого режиссера зима – самое продуктивное время года? 

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

# Submission

In [0]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '...', '2': '...'}

In [0]:
# и убедиться что ни чего не пропустил)
len(answers)